In [1]:
%run helper_functions.ipynb

In [3]:
import pandas as pd

import warnings

warnings.filterwarnings('ignore')

In [4]:
%%time

transactions = pd.read_parquet('data/transactions_train.parquet')
customers = pd.read_parquet('data/customers.parquet')
articles = pd.read_parquet('data/articles.parquet')

CPU times: total: 2.78 s
Wall time: 3.76 s


In [5]:
test_week = transactions.week.max()
transactions = transactions[transactions.week > transactions.week.max() - 10]

### Generating Candidates

#### Last Purchase candidates

In [6]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: total: 17.2 s
Wall time: 21.4 s


In [7]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: total: 484 ms
Wall time: 611 ms


In [8]:
candidates_last_purchase = transactions.copy()

In [9]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: total: 18.7 s
Wall time: 22.1 s


#### Bestsellers candidates

In [10]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [11]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [12]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [13]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [14]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [15]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [16]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [17]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

### Combining transactions and candidates / negative examples

In [18]:
transactions['purchased'] = 1

In [19]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

data.purchased.mean()

0.14606530579445656

In [20]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

#### Add bestseller information

In [21]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [22]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [23]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [24]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [25]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [26]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [27]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank']

In [28]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

CPU times: total: 344 ms
Wall time: 393 ms


In [29]:
test_baskets = test.groupby(['week', 'customer_id'])['article_id'].count().values

In [30]:
test_X = test[columns_to_use]
test_y = test['purchased']

### Model Training

In [31]:
from lightgbm.sklearn import LGBMRanker

In [32]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=3,
    importance_type='gain',
    verbose=10,
)

In [33]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
    eval_set=[(test_X, test_y)],
    eval_group=[test_baskets]
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.849315
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.152318
[LightGBM] [Debug] init for col-wise cost 0.187379 seconds, init for row-wise cost 0.520408 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.434930 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1079
[LightGBM] [Info] Number of data points in the train set: 10205052, number of used features: 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
CPU times: total: 26.3 s
Wall time: 14.3 s


In [34]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9849476818116674
article_id 0.007061542648568638
garment_group_no 0.0020872282760979045
product_type_no 0.00193149759931634
department_no 0.001342970697238775
colour_group_code 0.0007386998374111764
club_member_status 0.0005309456759098065
postal_code 0.0005161682919825599
graphical_appearance_no 0.00030249571439311954
Active 0.000280144870729268
index_group_no 0.00013516293283713886
age 0.0001182042839964077
perceived_colour_value_id 7.257359851502959e-06
fashion_news_frequency 0.0
FN 0.0
section_no 0.0
index_code 0.0
perceived_colour_master_id 0.0


In [35]:
ranker._best_score

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('ndcg@1', 0.9617024681901843),
                          ('ndcg@2', 0.9657799740565617),
                          ('ndcg@3', 0.9700073803424453),
                          ('ndcg@4', 0.9731790105361492),
                          ('ndcg@5', 0.9753736534831506)])})